In [ ]:
import sys
sys.path.append('..')
import cortex_etl as c_etl

ma = c_etl.analysis_initial_processing("../configs/3-ThalamicStimuli/3-ThalamicStimuli-MegaScan2Scalings.yaml") #, loglevel="ERROR"

In [2]:
a = ma.hex0_spikes
a_name = "hex0_spikes"
ma.analyzers[a_name]

In [3]:
# c_etl.post_analysis(a)
c_etl.plot_rasters(a)


----- Custom post analysis -----


[Parallel(n_jobs=36)]: Using backend LokyBackend with 36 concurrent workers.
2023-04-25 14:21:04,045 INFO blueetl.parallel: Considering 54 rows for columns ['simulation_id', 'circuit_id']
2023-04-25 14:21:04,048 INFO blueetl.parallel: Tasks to be executed: 54
[Parallel(n_jobs=36)]: Done   7 out of  54 | elapsed:    5.6s remaining:   37.3s
[Parallel(n_jobs=36)]: Done  13 out of  54 | elapsed:    5.8s remaining:   18.3s
[Parallel(n_jobs=36)]: Done  19 out of  54 | elapsed:    5.9s remaining:   10.9s
[Parallel(n_jobs=36)]: Done  25 out of  54 | elapsed:    6.0s remaining:    7.0s
[Parallel(n_jobs=36)]: Done  31 out of  54 | elapsed:    6.1s remaining:    4.5s
[Parallel(n_jobs=36)]: Done  37 out of  54 | elapsed:    6.2s remaining:    2.8s
[Parallel(n_jobs=36)]: Done  43 out of  54 | elapsed:    6.3s remaining:    1.6s
[Parallel(n_jobs=36)]: Done  49 out of  54 | elapsed:    6.3s remaining:    0.6s
[Parallel(n_jobs=36)]: Done  54 out of  54 | elapsed:    6.6s finished
[Parallel(n_jobs=36)]

----- Custom post analysis complete: 32.02s -----
Custom post analysis:  265
Custom post analysis:  865
Custom post analysis:  425
Custom post analysis:  185
Custom post analysis:  869
Custom post analysis:  509
Custom post analysis:  341
Custom post analysis:  29
Custom post analysis:  905
Custom post analysis:  229
Custom post analysis:  901
Custom post analysis:  261
Custom post analysis:  189
Custom post analysis:  673
Custom post analysis:  909
Custom post analysis:  665
Custom post analysis:  661
Custom post analysis:  421
Custom post analysis:  505
Custom post analysis:  269
Custom post analysis:  353
Custom post analysis:  233
Custom post analysis:  349
Custom post analysis:  65
Custom post analysis:  669
Custom post analysis:  385
Custom post analysis:  61
Custom post analysis:  705
Custom post analysis:  73
Custom post analysis:  701
Custom post analysis:  181
Custom post analysis:  709
Custom post analysis:  193
Custom post analysis:  741
Custom post analysis:  381
Custom po

In [10]:
# RATIOS PLOTS
######################################################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

def mean_normalise(data):
    normalised_data = data / np.mean(data)
    return normalised_data



def evoked_ratios_analaysis(a, neuron_classes):
    
    if ('vivo_ratios_df' in list(a.analysis_config.custom.keys())):
        
        vivo_evoked_ratios = pd.read_feather(a.analysis_config.custom['vivo_ratios_df']).etl.q(type='vivo', vivo=a.analysis_config.custom['vivo_ratios_dataset']) # Change to temp_ratios_df_path
        vivo_nc_ratios = vivo_evoked_ratios.etl.q(neuron_class=neuron_classes)['ratio']
        
        silico_evoked_ratios = pd.merge(a.features.mean_psth.df.reset_index(), a.features.max_psth.df.reset_index()).etl.q(window='evoked_SOZ_250ms')
        silico_evoked_ratios['ratio'] = silico_evoked_ratios['max_psth'] / silico_evoked_ratios['mean_psth']
        silico_evoked_ratios = pd.merge(silico_evoked_ratios, a.custom['sim_mask_df'], on=['simulation_id'])
        silico_evoked_ratios = silico_evoked_ratios.etl.q(SimBad100p50p25pDecayOverlySustained=False)
        
        silico_mean_ratios_across_sims = silico_evoked_ratios.groupby(['neuron_class']).median()['ratio']
        silico_ratio_std_across_sims = silico_evoked_ratios.groupby(['neuron_class']).std()['ratio']

        evoked_ratios_line_plot(a, silico_evoked_ratios, silico_mean_ratios_across_sims, vivo_nc_ratios, neuron_classes, normalisation_type='none')
        evoked_ratios_line_plot(a, silico_evoked_ratios, silico_mean_ratios_across_sims, vivo_nc_ratios, neuron_classes, normalisation_type='mean_normalise')

#         plt.figure(figsize=(8.2*0.25, 8.2*0.25))
#         for nc in neuron_classes:
#             kwargs = {}
#             if (not np.isnan(silico_ratio_std_across_sims.etl.q(neuron_class=nc).iloc[0])):
#                 kwargs["yerr"] = silico_ratio_std_across_sims.etl.q(neuron_class=nc)
#             plt.errorbar(vivo_evoked_ratios.etl.q(neuron_class=nc)['ratio'], silico_mean_ratios_across_sims.etl.q(neuron_class=nc), c=c_etl.LAYER_EI_NEURON_CLASS_COLOURS[nc], marker=c_etl.LAYER_EI_NEURON_CLASS_MARKERS[nc], **kwargs)
#         lims = [0.0, 200.0]
#         plt.gca().plot(lims, lims, 'k-', alpha=0.75, zorder=0)
#         plt.gca().set_xlim([0, 200.0])
#         plt.gca().set_ylim([0, 200.0])
#         plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(200))
#         plt.gca().xaxis.set_minor_locator(ticker.MultipleLocator(50))
#         plt.gca().yaxis.set_major_locator(ticker.MultipleLocator(200))
#         plt.gca().yaxis.set_minor_locator(ticker.MultipleLocator(50))
#         plt.gca().set_title('Evok/spont ratio')
#         plt.gca().set_xlabel('In-vivo', labelpad=-7.0)
#         plt.gca().set_ylabel('In-silico', labelpad=-7.0)
#         plt.tight_layout()
#         plt.savefig(str(a.figpaths.evoked) + "/EvokedRatios2.pdf")
#         plt.close()


#         if ("vpm_l5e_cond_scaling_factor" in silico_evoked_ratios.columns):
#             scaling_1_L5E_ratios = silico_evoked_ratios.etl.q(neuron_class="L5_EXC", vpm_l5e_cond_scaling_factor=1.0)
#             scaling_1_36_L5E_ratios = silico_evoked_ratios.etl.q(neuron_class="L5_EXC", vpm_l5e_cond_scaling_factor=1.36)
#             plt.figure(figsize=(8.2*0.25*0.75, 8.2*0.25))
#             plt.scatter([1.0 for i in range(len(scaling_1_L5E_ratios))], [scaling_1_L5E_ratios['ratio']], s=1, c='k')
#             plt.scatter([1.36 for i in range(len(scaling_1_36_L5E_ratios))], [scaling_1_36_L5E_ratios['ratio']], s=1, c='k')
#             for simulation_id in scaling_1_L5E_ratios['simulation_id'].unique():
#                 scaling_1_L5E_row = scaling_1_L5E_ratios.etl.q(simulation_id=simulation_id).iloc[0]
#                 scaling_1_36_L5E_same_vars = scaling_1_36_L5E_ratios.etl.q(ca=scaling_1_L5E_row['ca'], desired_connected_proportion_of_invivo_frs=scaling_1_L5E_row['desired_connected_proportion_of_invivo_frs'], depol_stdev_mean_ratio=scaling_1_L5E_row['depol_stdev_mean_ratio'], vpm_pct=scaling_1_L5E_row['vpm_pct'])
#                 if (len(scaling_1_36_L5E_same_vars)):
#                     plt.plot([1, 1.36], [scaling_1_L5E_row['ratio'], scaling_1_36_L5E_same_vars.iloc[0]['ratio']], c='k', lw='0.1')         

#             plt.gca().set_xlim([0.9, 1.46])
#             plt.gca().set_ylim([0.0, silico_evoked_ratios.etl.q(neuron_class="L5_EXC")['ratio'].max() * 1.05])
#             plt.gca().yaxis.set_major_locator(ticker.MultipleLocator(20))
#             plt.gca().yaxis.set_minor_locator(ticker.MultipleLocator(5))
#             plt.gca().set_ylabel('Evok/spont ratio', labelpad=-7.0)
#             plt.gca().set_xticks(ticks=[1.0, 1.36],labels=['Mean', 'Max'])
#             plt.gca().set_xlabel('TC -> L5E conductance')
#             plt.tight_layout()
#             plt.savefig(str(a.figpaths.evoked) + '/L5E_scaling_ratio_comparison.pdf')
#             plt.close()





def evoked_ratios_line_plot(a, silico_evoked_ratios, silico_mean_ratios_across_sims, vivo_nc_ratios, neuron_classes, normalisation_type='none'):

    plt.figure(figsize=(8.2*0.3, 8.2*0.3*0.35))
    plt.gca().set_facecolor('lightgrey')
    cmap = sns.color_palette("seismic", as_cmap=True)

    vivo_nc_mean_ratio = np.mean(vivo_nc_ratios)

    silico_mean_ratio_differences = []
    for simulation_id in silico_evoked_ratios.simulation_id.unique():
        simulation_nc_evoked_ratios = silico_evoked_ratios.etl.q(simulation_id=simulation_id, neuron_class=neuron_classes)
        
        if (normalisation_type == "none"):
            silico_mean_ratio_differences.append(np.mean(np.asarray(simulation_nc_evoked_ratios['ratio'])) - vivo_nc_mean_ratio)
        elif (normalisation_type == "mean_normalise"):
            silico_mean_ratio_differences.append(np.mean(mean_normalise(np.asarray(simulation_nc_evoked_ratios['ratio']))) - mean_normalise(vivo_nc_mean_ratio))


    if (normalisation_type == "none"):
        a.custom['all_window_stat_and_mask_df'].loc[a.custom['all_window_stat_and_mask_df'].etl.q(simulation_id=silico_evoked_ratios.simulation_id.unique()).index, "mean_ratio_difference"] = silico_mean_ratio_differences

    abs_max_silico_mean_ratio_differences = np.max(np.abs(silico_mean_ratio_differences))
    range_width = abs_max_silico_mean_ratio_differences * 2

    for i, simulation_id in enumerate(silico_evoked_ratios.simulation_id.unique()):

        simulation_nc_evoked_ratios = silico_evoked_ratios.etl.q(simulation_id=simulation_id, neuron_class=neuron_classes)
        c_val = (silico_mean_ratio_differences[i] / range_width) + 0.5

        if (normalisation_type == "none"):
            plt.plot(range(len(neuron_classes)), simulation_nc_evoked_ratios['ratio'], c=cmap(c_val), lw=.2)
        elif (normalisation_type == "mean_normalise"):
            plt.plot(range(len(neuron_classes)), mean_normalise(simulation_nc_evoked_ratios['ratio']), c='k', lw=.2)

    if (normalisation_type == "none"):
        plt.plot(range(len(neuron_classes)), vivo_nc_ratios, c='w', lw=1, marker='o', ms=3)
        plt.plot(range(len(neuron_classes)), silico_mean_ratios_across_sims.etl.q(neuron_class=neuron_classes), c='k', lw=1, marker='o', ms=3)
        plt.gca().set_ylim([0.0, 200.0])
        plt.gca().yaxis.set_major_locator(ticker.MultipleLocator(200))
        plt.gca().yaxis.set_minor_locator(ticker.MultipleLocator(50))

    elif (normalisation_type == "mean_normalise"):
        plt.plot(range(len(neuron_classes)), mean_normalise(vivo_nc_ratios), c='w', lw=1, marker='o', ms=3)
        plt.plot(range(len(neuron_classes)), mean_normalise(silico_mean_ratios_across_sims.etl.q(neuron_class=neuron_classes)), c='k', lw=1, marker='o', ms=3)
        plt.gca().set_ylim([0.0, 3.0])
        plt.gca().yaxis.set_major_locator(ticker.MultipleLocator(3.0))
        plt.gca().yaxis.set_minor_locator(ticker.MultipleLocator(1.0))



    
    plt.gca().set_xticks(range(len(neuron_classes)), labels=[c_etl.neuron_class_label_map[nc] for nc in neuron_classes])
    plt.gca().set_ylabel('Evok/\nspont', labelpad=-10.0)
    plt.tight_layout()
    plt.savefig(str(a.figpaths.evoked) + "/" + normalisation_type + "_EvokedRatios.pdf")

    plt.close()

c_etl.create_directories(a)

vivo_df, rp_psth_df, svo_psth_df = c_etl.preprocess_vivo_dfs()
neuron_classes = c_etl.select_comparison_neuron_classes(a)

c_etl.compare_vivo_and_silico(a, vivo_df)
c_etl.create_sim_mask(a, ["ALL"] + neuron_classes)
c_etl.merge_comparison_and_mask(a)
c_etl.filter_dfs_by_window(a)

evoked_ratios_analaysis(a, neuron_classes)
# create_heatmaps(a)
# compare_time_courses(a, vivo_df, silico_bin_size=0.5, silico_sigma=2)
# psth_plots(a, neuron_classes, rp_psth_df, svo_psth_df)

/nvme/isbister/1415817/ipykernel_105728/585100551.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  silico_mean_ratios_across_sims = silico_evoked_ratios.groupby(['neuron_class']).median()['ratio']
/nvme/isbister/1415817/ipykernel_105728/585100551.py:28: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  silico_ratio_std_across_sims = silico_evoked_ratios.groupby(['neuron_class']).std()['ratio']
/nvme/isbister/1415817/ipykernel_105728/585100551.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c